In [1]:
# ensure nltk is downloaded
import nltk
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [2]:
# peek at the content of the package
dir(nltk)

['AbstractLazySequence',
 'AffixTagger',
 'AlignedSent',
 'Alignment',
 'AnnotationTask',
 'ApplicationExpression',
 'Assignment',
 'BigramAssocMeasures',
 'BigramCollocationFinder',
 'BigramTagger',
 'BinaryMaxentFeatureEncoding',
 'BlanklineTokenizer',
 'BllipParser',
 'BottomUpChartParser',
 'BottomUpLeftCornerChartParser',
 'BottomUpProbabilisticChartParser',
 'Boxer',
 'BrillTagger',
 'BrillTaggerTrainer',
 'CFG',
 'CRFTagger',
 'CfgReadingCommand',
 'ChartParser',
 'ChunkParserI',
 'ChunkScore',
 'Cistem',
 'ClassifierBasedPOSTagger',
 'ClassifierBasedTagger',
 'ClassifierI',
 'ConcordanceIndex',
 'ConditionalExponentialClassifier',
 'ConditionalFreqDist',
 'ConditionalProbDist',
 'ConditionalProbDistI',
 'ConfusionMatrix',
 'ContextIndex',
 'ContextTagger',
 'ContingencyMeasures',
 'CoreNLPDependencyParser',
 'CoreNLPParser',
 'Counter',
 'CrossValidationProbDist',
 'DRS',
 'DecisionTreeClassifier',
 'DefaultTagger',
 'DependencyEvaluator',
 'DependencyGrammar',
 'DependencyGrap

## function we will use
pos_tag() allows us to tag speech
tokenize() splits a sentence into a list of words to facilitate parsing for python

In [3]:
from nltk.corpus import stopwords

stopwords.words('english')[0:500:20]

['i', 'himself', 'that', 'a', 'through', 'here', 'own', 're', 'ma']

## Structured vs Unstructured 
-Text data is mostl ikely unstructured. No delimiters. Email, social media posts.

In [1]:
# read in raw text
rawData = open(r"C:\Users\Loube\Desktop\Ex_Files_NLP_Python_ML_EssT\Exercise Files\Ch01\01_03\Start\SMSSpamCollection.tsv").read()
rawData[0:500] 

"ham\tI've been searching for the right words to thank you for this breather. I promise i wont take your help for granted and will fulfil my promise. You have been wonderful and a blessing at all times.\nspam\tFree entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's\nham\tNah I don't think he goes to usf, he lives around here though\nham\tEven my brother is not like to speak with me. They treat me like aid"

In [5]:
parsedData = rawData.replace("\t", "\n").split("\n")

In [6]:
parsedData[0:5]

['ham',
 "I've been searching for the right words to thank you for this breather. I promise i wont take your help for granted and will fulfil my promise. You have been wonderful and a blessing at all times.",
 'spam',
 "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's",
 'ham']

In [7]:
labelList = parsedData[0::2]
textList = parsedData[1::2]

In [8]:
print(labelList[0:5])
print("\n")
print(textList[0:5])

['ham', 'spam', 'ham', 'ham', 'ham']


["I've been searching for the right words to thank you for this breather. I promise i wont take your help for granted and will fulfil my promise. You have been wonderful and a blessing at all times.", "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's", "Nah I don't think he goes to usf, he lives around here though", 'Even my brother is not like to speak with me. They treat me like aids patent.', 'I HAVE A DATE ON SUNDAY WITH WILL!!']


In [9]:
#print(len(labelList))
#print(len(textList))
#labelList = labelList[:-1]
#print(labelList[-5:])

In [10]:
len(labelList)

5571

In [11]:
# paste labeled data into a dataframe as a dictionary
import pandas as pd

fullCorpus = pd.DataFrame({
    "label": labelList[:-1],
    "body_list": textList
})

fullCorpus[0:5]

,label,body_list
0,ham,I've been searching for the right words to tha...
1,spam,Free entry in 2 a wkly comp to win FA Cup fina...
2,ham,"Nah I don't think he goes to usf, he lives aro..."
3,ham,Even my brother is not like to speak with me. ...
4,ham,I HAVE A DATE ON SUNDAY WITH WILL!!


In [12]:
dataset = pd.read_csv("C://Users//Loube//Desktop//Ex_Files_NLP_Python_ML_EssT//Exercise Files//Ch01//01_03//Start//SMSSpamCollection.tsv", sep="\t", header=None)

In [13]:
fullCorpus.columns = ["label", "body_text"]
fullCorpus.head()

,label,body_text
0,ham,I've been searching for the right words to tha...
1,spam,Free entry in 2 a wkly comp to win FA Cup fina...
2,ham,"Nah I don't think he goes to usf, he lives aro..."
3,ham,Even my brother is not like to speak with me. ...
4,ham,I HAVE A DATE ON SUNDAY WITH WILL!!


## Explore dataset

In [14]:
# shape of dataset
print("Input data has {} rows and {} columns".format(len(fullCorpus), len(fullCorpus.columns)))

Input data has 5570 rows and 2 columns


In [15]:
# how many times spam and ham are repeated; filter on spam and ham
print("Out of {} rows, there are {} spam and {} ham".format(len(fullCorpus),
                                                           len(fullCorpus[fullCorpus['label']=="spam"]),
                                                            len(fullCorpus[fullCorpus['label']=="ham"])))

Out of 5570 rows, there are 746 spam and 4824 ham


In [16]:
# missing any data?
print("Number of nulls in label: {}".format(fullCorpus['label'].isnull().sum()))
print("Number of nulls in body_text: {}".format(fullCorpus['body_text'].isnull().sum()))

Number of nulls in label: 0
Number of nulls in body_text: 0


## Regex
'nlp' will return nlp
'[j-q]' will seek any alphabet between j and q
'[j-q0-9]'

In [17]:
import re

re_test = "This is a made up string to test 2 different regex methods"
re_test_messy = "This      is a made up        string to test 2    different regex methods"
re_test_messy1 = "This-is-a-made/up.string*to>>>>test----2""""""""different-regex-methods"

## Split a sentence into a list of words

In [18]:
# split a sentence into a list of words
re.split('\s', re_test)

['This',
 'is',
 'a',
 'made',
 'up',
 'string',
 'to',
 'test',
 '2',
 'different',
 'regex',
 'methods']

In [19]:
# split a sentence into a list of words
re.split('\s', re_test_messy)

['This',
 '',
 '',
 '',
 '',
 '',
 'is',
 'a',
 'made',
 'up',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'string',
 'to',
 'test',
 '2',
 '',
 '',
 '',
 'different',
 'regex',
 'methods']

In [20]:
# split a sentence into a list of words
re.split('\s+', re_test_messy)

['This',
 'is',
 'a',
 'made',
 'up',
 'string',
 'to',
 'test',
 '2',
 'different',
 'regex',
 'methods']

In [21]:
# split a sentence into a list of words
re.split('\s', re_test_messy1)

['This-is-a-made/up.string*to>>>>test----2different-regex-methods']

In [22]:
# split a sentence into a list of words
re.split('\W+', re_test_messy1)

['This',
 'is',
 'a',
 'made',
 'up',
 'string',
 'to',
 'test',
 '2different',
 'regex',
 'methods']

## Replace a specific string

In [23]:
pep8_test = "I try to follow PEP8 guidelines"
pep7_test = "I try to follow PEP7 guidelines"
peep8_test = "I try to follow PEEP 8 guidelines"

In [24]:
re.findall('\S+', re_test)

['This',
 'is',
 'a',
 'made',
 'up',
 'string',
 'to',
 'test',
 '2',
 'different',
 'regex',
 'methods']

In [25]:
re.findall('\S+', re_test_messy)

['This',
 'is',
 'a',
 'made',
 'up',
 'string',
 'to',
 'test',
 '2',
 'different',
 'regex',
 'methods']

In [26]:
re.findall('\S+', re_test_messy1)

['This-is-a-made/up.string*to>>>>test----2different-regex-methods']

In [27]:
re.findall('\w+', re_test_messy1) # \w seeks words as opposed to \W which seeks non-word

['This',
 'is',
 'a',
 'made',
 'up',
 'string',
 'to',
 'test',
 '2different',
 'regex',
 'methods']

## Replace strings

In [28]:
re.findall('[A-Z]+[0-9]+', peep8_test)

[]

In [29]:
re.sub('[A-Z][\s]+[0-9]+', "PEP8 Python Styleguide", peep8_test)

'I try to follow PEEPEP8 Python Styleguide guidelines'